# camera cat

Skin lesion classification of dermoscopic images using machine learning and convolutional neural network

19 December 2022

https://www.nature.com/articles/s41598-022-22644-9#Tab7

https://aihub.or.kr/aihubdata/data/view.do?currMenu=&topMenu=&aihubDataSe=realm&dataSetSn=561

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
import glob
import cv2
import numpy as np
from tqdm import tqdm

from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
from sklearn.preprocessing import LabelEncoder

from skimage.feature import graycomatrix

import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten, Conv2D, MaxPooling2D, Dropout, BatchNormalization, Concatenate

In [ ]:
class ImageLoader:
    def __init__(self, src_path, num_files_per_folder):
        self.src_path = src_path
        self.num_files_per_folder = num_files_per_folder

    def get_path_and_label(self):
        image_paths = []
        dermatological_lesions = []

        for root, dirs, files in tqdm(os.walk(self.src_path), desc='Walking directories', unit=' dir'):

            for dir in dirs:
                dir_path = os.path.join(root, dir)
                image_files = sorted(glob.glob(os.path.join(dir_path, '*.jpg')))

                for image_file in image_files[:self.num_files_per_folder]:
                    image_paths.append(image_file)
                    filename = os.path.basename(image_file)
                    parts = filename.split('_')
                    second_part = parts[2]
                    dermatological_lesions.append(second_part)

        return image_paths, dermatological_lesions

class ImagePreprocessor:
    def __init__(self, image_paths, target_size):
        self.image_paths = image_paths
        self.target_size = target_size

    def preprocess_images(self):
        images = []
        gray_images = []

        for image_path in tqdm(self.image_paths, desc='Preprocessing Images'):
            image = cv2.imread(image_path)

            if image is None:
                print(f"Error loading image: {image_path}")
                continue

            image = cv2.resize(image, self.target_size)

            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            gray_image = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)

            images.append(image)
            gray_images.append(gray_image)

        return np.array(images), np.array(gray_images)

class FeatureExtractor:
    def __init__(self, images, gray_images):
        self.images = images
        self.gray_images = gray_images

    def extract_color_histograms(self):
        color_histograms = np.zeros((self.images.shape[0], 512))

        for i, image in tqdm(enumerate(self.images), desc='Extracting Color Histograms'):
            histogram = cv2.calcHist([image], [0, 1, 2], None,[8 ,8 ,8 ], [0 ,256 ,0 ,256 ,0 ,256])
            histogram = cv2.normalize(histogram,histogram).flatten()
            color_histograms[i] = histogram

        return color_histograms

    def extract_hu_moments(self):
        moments_list = [cv2.HuMoments(cv2.moments(gray_image)).flatten() for gray_image in tqdm(self.gray_images)]
        hu_moments = np.array(moments_list)

        return hu_moments

    def extract_haralick_textures(self):
        textures_list = [np.mean(graycomatrix(gray_image, distances=[1], angles=[0], symmetric=True, normed=True), axis=(0, 1)).flatten() for gray_image in tqdm(self.gray_images, desc='Extracting Haralick Textures')]
        haralick_textures = np.array(textures_list)

        return haralick_textures

class DatasetCreator:
    def __init__(self, loader: ImageLoader,
                 preprocessor: ImagePreprocessor,
                 feature_extractor: FeatureExtractor):

        self.loader = loader
        self.preprocessor = preprocessor
        self.feature_extractor = feature_extractor

    def create_dataset(self):
        self.get_path_and_label()
        self.preprocess_images()
        self.extract_color_histograms()
        self.extract_hu_moments()
        self.extract_haralick_textures()

        self.feature_vectors = np.concatenate([self.color_histograms,
                                        self.hu_moments,
                                        self.haralick_textures], axis=1)

        encoder = LabelEncoder()
        encoded_labels = encoder.fit_transform(self.lesions)
        num_classes = len(encoder.classes_)
        one_hot_labels = tf.one_hot(encoded_labels, depth=num_classes)

        self.image = tf.convert_to_tensor(self.images, dtype=tf.float32)
        self.image = self.image / tf.constant(255.0, dtype=tf.float32)

        self.dataset = tf.data.Dataset.from_tensor_slices(({"image_input": self.image,
                                                    "global_feature_input": self.feature_vectors},
                                                    one_hot_labels))

class Pipeline:
    def __init__(self, loader: ImageLoader,
                 preprocessor: ImagePreprocessor,
                 feature_extractor: FeatureExtractor):

       self.loader = loader
       self.preprocessor = preprocessor
       self.feature_extractor = feature_extractor

    def execute(self):
       for image in self.loader.load_images():
           preprocessed_image = self.preprocessor.preprocess_image(image)
           features = self.feature_extractor.extract_features(preprocessed_image)
           yield features

In [ ]:
src_path = '/content/drive/Shareddrives/반려묘/일반카메라/Training'
num_files_per_folder = 2000
target_size = (256, 256)
buffer_size = 1000
batch_size = 64

image_paths, lesions = get_path_and_label(src_path, num_files_per_folder)

NameError: ignored

In [ ]:
image_feature_vectors = extract_features(image_paths)

Preprocessing Images: 100%|██████████| 20490/20490 [09:01<00:00, 37.85it/s]
Extracting Color Histograms: 20490it [00:03, 5848.99it/s]
Extracting Haralick Textures: 100%|██████████| 20490/20490 [00:08<00:00, 2517.30it/s]


In [ ]:
print('Number of image paths: ', len(image_paths))
print('Number of image feature vectors: ', len(image_feature_vectors))
print('Number of labels: ', len(lesions))

Number of image paths:  20490
Number of image feature vectors:  20490
Number of labels:  20490


In [ ]:
dataset = create_dataset(image_paths, image_feature_vectors, lesions)
dataset = dataset.shuffle(buffer_size).batch(batch_size)

In [ ]:
dataset_size = tf.data.experimental.cardinality(dataset).numpy()

train_ratio = 0.8
val_ratio = 0.1
test_ratio = 0.1

train_size = int(train_ratio * dataset_size)
val_size = int(val_ratio * dataset_size)
test_size = int(test_ratio * dataset_size)

train_dataset = dataset.take(train_size)
val_dataset_temp = dataset.skip(train_size)

val_dataset = val_dataset_temp.take(val_size)
test_dataset = val_dataset_temp.skip(val_size)

In [ ]:
print('Expected train_dataset size: ', train_size)
print('Expected val_dataset size: ', val_size)
print('Expected test_dataset size: ', test_size)

print('Actual train_dataset size: ', tf.data.experimental.cardinality(train_dataset).numpy())
print('Actual val_dataset size: ', tf.data.experimental.cardinality(val_dataset).numpy())
print('Actual test_dataset size: ', tf.data.experimental.cardinality(test_dataset).numpy())

Expected train_dataset size:  256
Expected val_dataset size:  32
Expected test_dataset size:  32
Actual train_dataset size:  256
Actual val_dataset size:  32
Actual test_dataset size:  33


In [ ]:
image_input = Input(shape=(256, 256, 3), name='image_input')
x = Conv2D(32, (3, 3), padding='same', activation='relu')(image_input)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(3 , 3))(x)

x = Dropout(0.25)(x)

x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(64, (3, 3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2 , 2))(x)

x = Dropout(0.25)(x)

x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling2D(pool_size=(2 , 2))(x)

x = Dropout(0.25)(x)

global_feature_input = Input(shape=(520,), name='global_feature_input')

combined_features= Concatenate()([Flatten()(x), global_feature_input])

combined_features= Dense(1024, activation='relu')(combined_features)
combined_features= BatchNormalization ()(combined_features)
combined_features= Dropout(0.5)(combined_features)

outputs= Dense(4, activation='softmax')(combined_features)

model= Model(inputs=[image_input, global_feature_input], outputs=outputs)

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=150,
    batch_size=64
)

Epoch 1/150
219/256 [========================>.....] - ETA: 9s - loss: 1.5696 - accuracy: 0.5117

InvalidArgumentError: ignored

In [ ]:
y_true = np.concatenate([y.numpy() for x, y in test_dataset], axis=0)
y_pred = model.predict(test_dataset)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true_classes = np.argmax(y_true, axis=1)

print(classification_report(y_true_classes, y_pred_classes))

print(confusion_matrix(y_true_classes, y_pred_classes))

roc_auc = roc_auc_score(y_true, y_pred , multi_class='ovr')
print('ROC-AUC score:', roc_auc)

In [ ]:
# TensorFlow Lite 모델로 변환
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# 변환된 모델을 파일로 저장
with open('3000_256.tflite', 'wb') as f:
    f.write(tflite_model)

# microscope cat

Intelligent diagnostic model for malaria parasite detection and classification using imperative inception-based capsule neural networks

17 August 2023

https://www.nature.com/articles/s41598-023-40317-z#Sec3

In [ ]:
from tensorflow.keras.layers import PReLU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.layers import Concatenate, MaxPooling2D, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import concatenate
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.losses import mean_squared_error

def inception_module(input_layer, filters):
    conv1x1 = Conv2D(filters[0], (1, 1), activation='relu')(input_layer)
    conv3x3_reduce = Conv2D(filters[1], (1, 1), activation='relu')(input_layer)
    conv3x3 = Conv2D(filters[2], (3, 3), padding='same', activation='relu')(conv3x3_reduce)
    conv5x5_reduce = Conv2D(filters[3], (1, 1), activation='relu')(input_layer)
    conv5x5 = Conv2D(filters[4], (5, 5), padding='same', activation='relu')(conv5x5_reduce)
    maxpool = MaxPooling2D((3, 3), strides=(1, 1), padding='same')(input_layer)
    maxpool_conv = Conv2D(filters[5], (1, 1), activation='relu')(maxpool)
    inception_output = Concatenate(axis=-1)([conv1x1, conv3x3, conv5x5, maxpool_conv])
    return inception_output

# Input
input_shape = (128, 128, 3)
input_layer = Input(shape=input_shape)

# Inception block
inception_output = inception_module(input_layer, filters=[64, 128, 192, 32, 96, 64])
inception_output = inception_module(inception_output, filters=[64, 128, 192, 32, 96, 64])
# Add more inception modules if needed

# Primary Capsule layer
primary_capsules = Conv2D(32, (1, 1), activation='relu')(inception_output)

# Higher Capsule layers
# (Add imperative routing mechanism layers here)

# PReLU activation for routing
higher_capsules_prelu = PReLU()(higher_capsules)

# Flatten and Fully Connected layers
capsule_flatten = Flatten()(higher_capsules_prelu)  # Flatten higher capsules
output_layer = Dense(2, activation='softmax')(capsule_flatten)  # Two capsules: parasitized and uninfected

# Create the model
model = Model(inputs=input_layer, outputs=output_layer)

# Compile the model with Adam optimizer and custom loss function
optimizer = Adam(learning_rate=0.007, beta_1=0.8)
loss_fn = custom_loss_function # Define the custom loss function as described in the paper
model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])

# Print the model summary
model.summary()

# symptom classification

In [1]:
import os
import multiprocessing

import glob
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import torch
from torch import nn, optim
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import models
from torchvision import transforms
from torchvision.io import read_image

In [2]:
class ImageLoader:
    def __init__(self, src_path, num_files_per_folder):
        self.src_path = src_path
        self.num_files_per_folder = num_files_per_folder

    def get_image_paths_and_labels(self):
        image_paths = []
        symptom_labels = []

        for root, dirs, files in os.walk(self.src_path):
            for dir in dirs:
                dir_path = os.path.join(root, dir)
                image_files = sorted(glob.glob(os.path.join(dir_path, '*.jpg')))

                for image_file in image_files[:self.num_files_per_folder]:
                    image_paths.append(image_file)
                    filename = os.path.basename(image_file)
                    symptom_labels.append(0 if "A7" in filename else 1)

        return image_paths, symptom_labels

class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self,idx):
        try:
            img = read_image(self.image_paths[idx])
            img = img.float() / 255
            img = self.transform(img)
            label = torch.tensor(int(self.labels[idx]),dtype=torch.long)
            return img,label

        except Exception as e:
            print(f"Error loading image: {self.image_paths[idx]}, {str(e)}")

class DataPipeline:
    def __init__(self, src_path, num_files_per_folder, target_size, batch_size):
        loader = ImageLoader(src_path=src_path, num_files_per_folder=num_files_per_folder)
        self.image_paths, self.labels = loader.get_image_paths_and_labels()

        data_transforms=transforms.Compose([transforms.Resize(target_size, antialias=True)])
        dataset = CustomDataset(self.image_paths, self.labels, transform=data_transforms)

        print(f"Size of the Dataset: {len(dataset)}")

        total_size = len(dataset)
        train_size = int(0.7 * total_size)
        valid_size = int(0.15 * total_size)
        test_size = total_size - train_size - valid_size

        self.train_dataset, self.valid_dataset, self.test_dataset = random_split(dataset, [train_size, valid_size, test_size])

        self.train_dataloader = DataLoader(self.train_dataset,
                                      batch_size=batch_size,
                                      shuffle=True,
                                      num_workers=multiprocessing.cpu_count())

        self.valid_dataloader = DataLoader(self.valid_dataset,
                                      batch_size=batch_size,
                                      num_workers=multiprocessing.cpu_count())

        self.test_dataloader = DataLoader(self.test_dataset,
                                     batch_size=batch_size,
                                     num_workers=multiprocessing.cpu_count())

    def get_dataloader(self):
        for dataloader in [self.train_dataloader, self.valid_dataloader, self.test_dataloader]:
            if isinstance(dataloader, torch.utils.data.DataLoader):
                print(f"{dataloader} creation successful")
            else:
                print(f"{dataloader} creation failed")

            try:
                images, labels = next(iter(dataloader))
                print("Batch data retrieval successful")
                print(f"Images shape: {images.shape}")
                print(f"Labels shape: {labels.shape}")
            except Exception as e:
                print("Batch data retrieval failed:", str(e))

        return self.train_dataloader, self.valid_dataloader, self.test_dataloader

In [3]:
pipeline = DataPipeline(src_path="/content/drive/Shareddrives/반려묘/일반카메라",
                        num_files_per_folder=500,
                        target_size=(256, 256),
                        batch_size=64)

train_dataloader, valid_dataloader, test_dataloader = pipeline.get_dataloader()

Size of the Dataset: 5500
<torch.utils.data.dataloader.DataLoader object at 0x78b28758bb50> creation successful
Batch data retrieval successful
Images shape: torch.Size([64, 3, 256, 256])
Labels shape: torch.Size([64])
<torch.utils.data.dataloader.DataLoader object at 0x78b2878754e0> creation successful
Batch data retrieval successful
Images shape: torch.Size([64, 3, 256, 256])
Labels shape: torch.Size([64])
<torch.utils.data.dataloader.DataLoader object at 0x78b28758bb20> creation successful
Batch data retrieval successful
Images shape: torch.Size([64, 3, 256, 256])
Labels shape: torch.Size([64])


In [ ]:
class ImageClassifier:
    def __init__(self, num_epochs, early_stopping_patience):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.model = models.resnet50(pretrained=True)
        num_ftrs = self.model.fc.in_features
        self.model.fc = nn.Linear(num_ftrs, 2)
        self.model.to(self.device)

        if torch.cuda.device_count() > 1:
            print(f"Let's use {torch.cuda.device_count()} GPUs!")
            self.model = nn.DataParallel(self.model)

        self.criterion = nn.CrossEntropyLoss()
        self.optimizer_ft = torch.optim.AdamW(self.model.parameters(), lr=0.001)

        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer_ft, step_size=7, gamma=0.1)

        self.num_epochs = num_epochs
        self.patience = early_stopping_patience

    def train_and_validate(self, dataloader, is_train):
        total_loss = 0.0
        total_corrects = 0

        for inputs, labels in dataloader:
            inputs, labels = inputs.to(self.device), labels.to(self.device)

            if is_train:
                self.model.train()
                torch.set_grad_enabled(True)
            else:
                self.model.eval()
                torch.set_grad_enabled(False)

            outputs = self.model(inputs)
            _, preds = torch.max(outputs, dim=1)

            loss = self.criterion(outputs, labels)

            if is_train:
                loss.backward()
                self.optimizer_ft.step()

            total_loss += loss.item() * inputs.size(0)
            total_corrects += torch.sum(preds == labels.data).item()

        epoch_loss = total_loss / len(dataloader.dataset)
        epoch_accuarcy = total_corrects / len(dataloader.dataset)

        return epoch_loss, epoch_accuarcy

    def train(self, train_dataloader, valid_dataloader):

        best_model_wts = copy.deepcopy(self.model.state_dict())
        best_accuracy_val_score = -1

        patience_counter = 0

        for epoch in range(self.num_epochs):

            train_loss ,train_accuracy=self.train_and_validate(train_dataloader,is_train=True)
            val_loss,val_accuracy=self.train_and_validate(valid_dataloader,is_train=False)

            print(f"Epoch [{epoch+1}/{self.num_epochs}] Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.4f} Valid Loss: {val_loss:.4f}, Valid Acc: {val_accuracy:.4f}")

            if val_accuracy > best_accuracy_val_score :
                best_accuracy_val_score=val_accuracy
                patience_counter=0
                best_model_wts=self.model.state_dict()

            else :
                patience_counter+=1

            if patience_counter>self.patience :
                print('Early Stopping!')
                break

        self.model.load_state_dict(best_model_wts)

    def test(self, test_dataloader):
        y_true_classes=[]
        y_pred_classes=[]
        y_pred_proba=[]

        with torch.no_grad():
            for inputs, labels in test_dataloader:
                inputs, labels = inputs.to(self.device), labels.to(self.device)

                outputs = self.model(inputs)
                probas = torch.nn.functional.softmax(outputs, dim=1)

                _, preds = torch.max(probas, dim=1)

                y_true_classes.extend(labels.detach().cpu().numpy())
                y_pred_classes.extend(preds.cpu().numpy())

                y_pred_proba.extend(probas[:, 1].cpu().numpy())

        print(classification_report(y_true_classes,y_pred_classes))
        print(confusion_matrix(y_true_classes,y_pred_classes))

        roc_auc = roc_auc_score(y_true_classes, y_pred_proba)
        print('ROC-AUC score:', roc_auc)

In [5]:
net = ImageClassifier(num_epochs=150, early_stopping_patience=5)
net.train(train_dataloader, valid_dataloader)

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 192MB/s]


Epoch [1/150] Train Loss: 0.5566, Train Acc: 0.7475 Valid Loss: 0.5528, Valid Acc: 0.7248
Epoch [2/150] Train Loss: 0.4963, Train Acc: 0.7721 Valid Loss: 0.5272, Valid Acc: 0.7552
Epoch [3/150] Train Loss: 0.4668, Train Acc: 0.7945 Valid Loss: 0.5077, Valid Acc: 0.7770
Epoch [4/150] Train Loss: 0.4506, Train Acc: 0.8015 Valid Loss: 0.5484, Valid Acc: 0.7612
Epoch [5/150] Train Loss: 0.4264, Train Acc: 0.8103 Valid Loss: 0.5484, Valid Acc: 0.7867
Epoch [6/150] Train Loss: 0.4331, Train Acc: 0.8137 Valid Loss: 0.4980, Valid Acc: 0.7745
Epoch [7/150] Train Loss: 0.4043, Train Acc: 0.8293 Valid Loss: 0.5049, Valid Acc: 0.7891
Epoch [8/150] Train Loss: 0.3658, Train Acc: 0.8420 Valid Loss: 0.5172, Valid Acc: 0.7709
Epoch [9/150] Train Loss: 0.3398, Train Acc: 0.8563 Valid Loss: 0.5090, Valid Acc: 0.7988
Epoch [10/150] Train Loss: 0.3468, Train Acc: 0.8491 Valid Loss: 0.5297, Valid Acc: 0.7733
Epoch [11/150] Train Loss: 0.3372, Train Acc: 0.8561 Valid Loss: 0.5677, Valid Acc: 0.7842
Epoch [1

In [6]:
net.test(test_dataloader)

              precision    recall  f1-score   support

           0       0.80      0.76      0.78       483
           1       0.69      0.74      0.71       343

    accuracy                           0.75       826
   macro avg       0.75      0.75      0.75       826
weighted avg       0.76      0.75      0.75       826

[[369 114]
 [ 90 253]]
ROC-AUC score: 0.8095660624498247


In [8]:
torch.save(net.model, "/content/drive/MyDrive/Classifier.pth")

In [11]:
class ModelPredictor:
    def __init__(self, model_path, target_size):
        self.model = torch.load(model_path)
        self.model.eval()
        self.target_size = target_size

    def preprocess(self, image_path):
        img = read_image(image_path)
        img = img.float() / 255
        transform = transforms.Compose([transforms.Resize(self.target_size, antialias=True)])
        img = transform(img)
        img_tensor = img.unsqueeze(0).to(next(self.model.parameters()).device)

        return img_tensor

    def predict(self, image_path):
        with torch.no_grad():
            input_tensor = self.preprocess(image_path)

            output_tensor = self.model(input_tensor)

            probabilities = torch.nn.functional.softmax(output_tensor[0], dim=0)

            predicted_class_index = torch.argmax(probabilities).item()

            return predicted_class_index, probabilities[predicted_class_index].item()

In [12]:
predictor=ModelPredictor(model_path="/content/drive/MyDrive/Classifier.pth", target_size=(256,256))
class_idx, probability=predictor.predict("/content/drive/MyDrive/IMG_C_A6_807768.jpg")
print(f"Predicted class: {class_idx}, Probability: {probability}")

Predicted class: 0, Probability: 0.9960675239562988
